In [3]:
from __future__ import unicode_literals, print_function
import ast 
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import time
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [5]:
df.head()

,review_id,text,entities
0,yi0R0Ugj_xUx_Nek0-_Qig,Went in for a lunch. Steak sandwich was delici...,"[(66, 71, 'PRODUCT'), (219, 224, 'PRODUCT'), (..."
1,fdiNeiN_hoCxCMy2wTRW9g,I'll be the first to admit that I was not exci...,"[(1145, 1152, 'PRODUCT'), (1216, 1223, 'PRODUC..."
2,G7XHMxG0bx9oBJNECG4IFg,Tracy dessert had a big name in Hong Kong and ...,"[(900, 907, 'PRODUCT'), (922, 925, 'PRODUCT'),..."
3,8e9HxxLjjqc9ez5ezzN7iQ,This place has gone down hill. Clearly they h...,"[(130, 134, 'PRODUCT'), (238, 243, 'PRODUCT'),..."
4,kbtscdyz6lvrtGjD1quQTg,"Like walking back in time, every Saturday morn...","[(422, 427, 'PRODUCT'), (349, 352, 'PRODUCT'),..."


In [ ]:
# read in subset
import pandas
import random

filename = "./spacy_train_clean.csv"
n = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)

In [17]:
s = 10000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pandas.read_csv(filename, skiprows=skip)

In [18]:
df.to_csv("spacy_train_clean_10k.csv")

In [12]:
df.head()

,review_id,text,entities
0,iVBDIRqz4-8xs8nJ8qQVUw,Amazing pizza!!! Although they did forget my c...,"[(8, 13, 'PRODUCT'), (45, 51, 'PRODUCT')]"
1,ZVIyILe9mr9JfAb-_ADzLg,The Rail does a lot of things right. The food ...,"[(255, 262, 'PRODUCT'), (41, 45, 'PRODUCT'), (..."
2,W5r9x027dszPMnCtnSWaJw,I love love their princess chicken dish. The c...,"[(27, 34, 'PRODUCT'), (45, 50, 'PRODUCT'), (14..."
3,0H_tEaI4mpRZkCl6NiDLAw,Another buffet on The Strip. Definitely bette...,"[(378, 388, 'PRODUCT'), (8, 14, 'PRODUCT'), (9..."
4,GV2I0JZwOvFuhdRlUY2C_w,Came in last week celebrating my bday. We walk...,"[(505, 512, 'PRODUCT'), (284, 293, 'PRODUCT'),..."


In [19]:
df = pd.read_csv("spacy_train_clean_10k.csv")
df['entities_clean']=[ast.literal_eval(i) for i in df['entities']]
#train_df, test_df = train_test_split(df, test_size = .2)
all_train, therest = train_test_split(df, train_size=1000)
train_df, test_df = train_test_split(all_train, test_size=.2)

def create_train_data(df):
  train_data = []
  newnlp = spacy.load("en_core_web_sm")

  for i in range(len(df)):
    doc = newnlp(df['text'].iloc[i])
    entity_list = df['entities_clean'].iloc[i]
    for ent in doc.ents:
      entity_list.append((ent.start_char, ent.end_char, ent.label_))
    entity_dict = {"entities": entity_list}
    train_data.append((df['text'].iloc[i], entity_dict))
  return train_data

def create_test_data(df):
  test_data = []
  newnlp = spacy.load("en_core_web_sm")

  for i in range(len(df)):
    doc = newnlp(df['text'].iloc[i])
    entity_list = df['entities_clean'].iloc[i]
    for ent in doc.ents:
      entity_list.append((ent.start_char, ent.end_char, ent.label_))
    entity_dict = {"entities": entity_list}
    test_data.append((df['text'].iloc[i], entity_dict))
  return test_data

# new entity label
LABEL = "PRODUCT"

TRAIN_DATA = create_train_data(train_df)
TEST_DATA = create_test_data(test_df)

# new entity label
def train(train_data=TRAIN_DATA, test_data=TEST_DATA, model='en_core_web_sm', new_model_name="product", output_dir='./ermodel', n_iter=1):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(LABEL)  # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch the examples using spaCy's minibatch
        start = time.time()
        for itn in range(n_iter):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)            
            #print("Training Recall:",nlp.evaluate(random.sample(TRAIN_DATA,200)).ents_r)
            #print("Test Recall:",nlp.evaluate(TEST_DATA).ents_p) #COMMENT: isn't this precision?
            #COMMENT: so test data here is evaluating test_data which has the format 
            # of e.g. ("Uber blew through $1 million a week", {"entities": [(0, 4, "ORG")]}) right
            #print("Training Losses", losses)
        end = time.time()
    print("Total training time:",end-start)

    # test the trained model (small sample test)
    for i in range(10):
      test_text = test_data[i][0]
      doc = nlp(test_text)
      print("Entities in '%s'" % test_text)
      for ent in doc.ents:
          print(ent.label_, ent.text)

    # COMMENT: Abstract to another function
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # COMMENT: Abstract to another function 
        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)
    return nlp

model = train()

Loaded model 'en_core_web_sm'
Total training time: 353.90298986434937
Entities in 'I am sad to report that my second visit was not as amazing as my first. I really hope this was just an off night at Public School 702, because I really love the ambiance and live music on weekend nights. 

I was feeling like pasta with a meaty red sauce, so I ordered the bolognese. But it had a very strange flavor and atypical meat sauce. I think the only reason I ate it without complaint was due to the utter mortifying food experience and all around bad day my friend was having.

The chef made things right though by making me an off the menu light salad. The poached pears with mixed greens and prosciutto salad topped with candied pecans and goat cheese was really really good!! I would come back just to have this salad.

For dessert, I had the lemon meringue with a graham cracker bottom. The dessert looked absolutely scrumptious. The meringue was super good, but the lemon custard was way too sweet for my

GPE Vegas
DATE this past weekend
ORDINAL 23rd
PERSON Earl
DATE Saturday
QUANTITY around 12pm
PRODUCT food
PRODUCT sandwich
WORK_OF_ART The Full Montagu
PRODUCT beef
PRODUCT lettuce
PRODUCT sauce
PRODUCT water
ORG NAO
PERSON Earl
PRODUCT bread
NORP Cheese
PRODUCT stringy
PRODUCT sauce
PRODUCT sandwich
PRODUCT table
PRODUCT bit
NORP Hawaiian
PRODUCT sandwich
ORDINAL first
GPE Sandwich
PERSON Earl
DATE next weekend
GPE Vegas
PRODUCT place
Saved model to ermodel
Loading from ermodel
PERSON Earl
QUANTITY 264.8 miles
PERSON Earl
DATE the day
GPE Vegas
PERSON Earl
DATE last year
PERSON Earl
GPE Sandwich
GPE Vegas
PRODUCT sandwich
PERSON Earl
GPE Vegas
DATE this past weekend
ORDINAL 23rd
PERSON Earl
DATE Saturday
QUANTITY around 12pm
PRODUCT food
PRODUCT sandwich
WORK_OF_ART The Full Montagu
PRODUCT beef
PRODUCT lettuce
PRODUCT sauce
PRODUCT water
ORG NAO
PERSON Earl
PRODUCT bread
NORP Cheese
PRODUCT stringy
PRODUCT sauce
PRODUCT sandwich
PRODUCT table
PRODUCT bit
NORP Hawaiian
PRODUCT sandwic

In [20]:
model